In [1]:
import sys
import os
import cobra
import libsbml
from cobra import Model, Reaction, Metabolite, manipulation
import pandas as pd
import csv
import copy
from pprint import pprint



#Change working dir first, ty ChatGPT, much loves
cwd = os.getcwd()
# Split the path into a list of directories
directories = cwd.split(os.sep)
# Remove the last two directories from the list
directories = directories[:-2]
# Join the directories back into a path
new_cwd = os.sep.join(directories)
# Change the current working directory to the new path
os.chdir(new_cwd)

sys.path.append("./src")

import model_manipulation  as mm



In [2]:
#These functions are used just to fix the ids in the legacy SBML, which wasn't fixed by CobraPy's SBML module.
#This would at least allow me to start my pipeline from actual scratch.
_renames = (
    (".", "_DOT_"),
    ("(", "_LPAREN_"),
    (")", "_RPAREN_"),
    ("-", "_DASH_"))

id_to_fix = "EX_co2_LPAREN_e_RPAREN__BS"



def escape_str_id_new(id_str):
    """make a single string id SBML compliant"""
    for c in ("'", '"'):
        if id_str.startswith(c) and id_str.endswith(c) \
                and id_str.count(c) == 2:
            id_str = id_str.strip(c)
    for char, escaped_char in _renames:
        id_str = id_str.replace(escaped_char, char)
    return id_str


In [3]:
%%capture

ios2164_orig = cobra.io.read_sbml_model("./Supplementary_PrevLit/Lakshmanan et al (2015)/ios2164.xml");

ios2164_orig.solver = 'gurobi'

inf = 1e6
#Set all non-zero flux bounds to 1e-6

mm.set_flux_to_1e6(ios2164_orig)

#Fix reaction & metabolite names
for rxns in ios2164_orig.reactions:
    id_old = rxns.id
    rxns.id = escape_str_id_new(id_old)
    
for mets in ios2164_orig.metabolites:
    id_old = mets.id
    mets.id = escape_str_id_new(id_old)
    


In [4]:
#Change PEPC stoichiometry from consuming CO2 to consuming HCO3
co2_c = ios2164_orig.metabolites.get_by_id('co2_c')
hco3_c = ios2164_orig.metabolites.get_by_id('hco3_c')
h_c = ios2164_orig.metabolites.get_by_id('h_c')
h2o_c = ios2164_orig.metabolites.get_by_id('h2o_c')

PEPC_old = ios2164_orig.reactions.PPCc
PEPC_temp = PEPC_old

# # #Removes old co2, h and h2o in favor of hco3
PEPC_temp.add_metabolites({hco3_c:-1, co2_c:1, h2o_c:1, h_c:-1})

ios2164_orig.reactions.PPCc = PEPC_temp

#PEPC now mass-balanced after adjusting coefficients
ios2164_orig.reactions.PPCc.check_mass_balance()

{}

In [5]:
#let's add a reaction specific for phloem, which includes only amino acids, starch and sucrose
#Values are from Poolman et al. (2014) and is rescaled from mol/gcw * time to mmol/gcw * time (similar to ios2164 and Arnold's Atmodel)


#Read from csv
read_phl = pd.read_csv('./misc/phloem_sap_comp.csv')
phl_dict = dict(read_phl.values)


model_rxn_list = list()
for items in phl_dict:
    #Update reaction signs first so it's all negative 
    phl_dict[items] = phl_dict[items] * -1
    
    #Retrieve pointer from original model to retrieve metabolite
    model_rxn = ios2164_orig.metabolites.get_by_id(items)
    model_rxn_list.append(model_rxn)

#Update rxns to pointers to orig model
for rxns in model_rxn_list:
    phl_dict[rxns] = phl_dict.pop(rxns.id)

#Generate new phloem reaction
phl_rxn = Reaction('DM_Phloem')
phl_rxn.name = 'export reaction for phloem sap'
phl_rxn.add_metabolites(phl_dict)
phl_rxn.notes['SUBSYSTEM']='Demand'



ios2164_orig.add_reactions([phl_rxn])
    

In [6]:
#Add NGAM reactions based on B&B's measurements
#This is a  good way to account for metabolic costs. However, the only limitation would this be would be that the reaction bounds are constant and non-variable

ngam_atp_c = Reaction('ngam_atp_c')
ngam_atp_s = Reaction('ngam_atp_s')
ngam_atp_x = Reaction('ngam_atp_x')
ngam_atp_m = Reaction('ngam_atp_m')

#Set names
ngam_atp_c.name = 'ATP NGAM for Cytosol'
ngam_atp_s.name = 'ATP NGAM for Chloroplast (Plastid)'
ngam_atp_x.name = 'ATP NGAM for Peroxisome'
ngam_atp_m.name = 'ATP NGAM for Mitochondria'

#Retrieve metabolites per compartment
atp_c = ios2164_orig.metabolites.atp_c
adp_c = ios2164_orig.metabolites.adp_c
h2o_c = ios2164_orig.metabolites.h2o_c
pi_c = ios2164_orig.metabolites.pi_c
h_c = ios2164_orig.metabolites.h_c

atp_s = ios2164_orig.metabolites.atp_s
adp_s = ios2164_orig.metabolites.adp_s
h2o_s = ios2164_orig.metabolites.h2o_s
pi_s = ios2164_orig.metabolites.pi_s
h_s = ios2164_orig.metabolites.h_s

atp_x = ios2164_orig.metabolites.atp_x
adp_x = ios2164_orig.metabolites.adp_x
h2o_x = ios2164_orig.metabolites.h2o_x
pi_x = ios2164_orig.metabolites.pi_x
h_x =  ios2164_orig.metabolites.h_x

atp_m = ios2164_orig.metabolites.atp_m
adp_m = ios2164_orig.metabolites.adp_m
h2o_m = ios2164_orig.metabolites.h2o_m
pi_m = ios2164_orig.metabolites.pi_m
h_m = ios2164_orig.metabolites.h_m




ngam_atp_c.add_metabolites({atp_c:-1, h2o_c: -1, adp_c:1, pi_c:1, h_c: 1})
ngam_atp_s.add_metabolites({atp_s:-1, h2o_s: -1, adp_s:1, pi_s:1, h_s: 1})
ngam_atp_x.add_metabolites({atp_x:-1, h2o_x: -1, adp_x:1, pi_x:1, h_x: 1})
ngam_atp_m.add_metabolites({atp_m:-1, h2o_m: -1, adp_m:1, pi_m:1, h_m: 1})

#Fix bounds of NGAM reaction. Values are from B&B. 
#However, what I'll implement instead will be Topfer's linear NGAM approach, which scales NGAM with PPFD

ngam_atp_c.notes['SUBSYSTEM']='Maintainance reaction'
ngam_atp_s.notes['SUBSYSTEM']='Maintainance reaction'
ngam_atp_x.notes['SUBSYSTEM']='Maintainance reaction'
ngam_atp_m.notes['SUBSYSTEM']='Maintainance reaction'

ios2164_orig.add_reactions([ngam_atp_c])
ios2164_orig.add_reactions([ngam_atp_s])
ios2164_orig.add_reactions([ngam_atp_x])
ios2164_orig.add_reactions([ngam_atp_m])

#Change bounds of NGAM reaction to one way only.
ngam_atp_c.bounds = (0,inf) #0.0427
ngam_atp_s.bounds = (0,inf) #0.1527
ngam_atp_x.bounds = (0,inf) #0.0076
ngam_atp_m.bounds = (0,inf) #0.0091

In [7]:
#NGAM for NADPH should be defined in the following compartments:

#cytosol, mitochondria and plastid

ngam_nadphox_c = Reaction('ngam_nadphox_c')
ngam_nadphox_s = Reaction('ngam_nadphox_s')
ngam_nadphox_m = Reaction('ngam_nadphox_m')

ngam_nadphox_c.name = "Generic cytosolic NADPH Oxidase for NGAM"
ngam_nadphox_s.name = "Generic stromal NADPH Oxidase for NGAM"
ngam_nadphox_m.name = "Generic mitochondrial NADPH Oxidase for NGAM"

#reaction would be the following
# #NADPH + 2O2 ↔ NADP+ + 2O2− + H+
#From Cheung et al (2013)'s work on modelling the effects of NGAM on plant fluxes, which in turn was  followed 
# NADP_c + x_NADPHWorkc + WATER_c ⇔ NADPH_c + ×met_2259 + PROTON_c	
#Met_2259 is oxygen

#Get all the constituent metabolites for the generic NADPH oxidase reaction in each compartment
nadph_c = ios2164_orig.metabolites.nadph_c
nadph_s = ios2164_orig.metabolites.nadph_s
nadph_m = ios2164_orig.metabolites.nadph_m


nadp_c = ios2164_orig.metabolites.nadp_c
nadp_s = ios2164_orig.metabolites.nadp_s
nadp_m = ios2164_orig.metabolites.nadp_m

h_c = ios2164_orig.metabolites.h_c
h_s = ios2164_orig.metabolites.h_s
h_m = ios2164_orig.metabolites.h_m

o2_c = ios2164_orig.metabolites.o2_c
o2_s = ios2164_orig.metabolites.o2_s
o2_m = ios2164_orig.metabolites.o2_m

h2o2_c = ios2164_orig.metabolites.h2o2_c
h2o2_s =  ios2164_orig.metabolites.h2o2_s
h2o2_m =  ios2164_orig.metabolites.h2o2_m


ngam_nadphox_c.add_metabolites({nadph_c:-1, o2_c:-1, h_c:-1, nadp_c:1, h2o2_c: 1})
ngam_nadphox_s.add_metabolites({nadph_s:-1, o2_s:-1, h_s:-1, nadp_s:1, h2o2_s: 1})
ngam_nadphox_m.add_metabolites({nadph_m:-1, o2_m:-1, h_m:-1, nadp_m:1, h2o2_m: 1})

ngam_nadphox_c.bounds=(0, inf)
ngam_nadphox_s.bounds=(0,inf)
ngam_nadphox_m.bounds=(0,inf)

ngam_nadphox_c.notes['SUBSYSTEM']='Maintainance reaction'
ngam_nadphox_s.notes['SUBSYSTEM']='Maintainance reaction'
ngam_nadphox_m.notes['SUBSYSTEM']='Maintainance reaction'

ios2164_orig.add_reactions([ngam_nadphox_c])
ios2164_orig.add_reactions([ngam_nadphox_s])
ios2164_orig.add_reactions([ngam_nadphox_m])

#Notes: Nov 22, 2022 -- Changed NADPHOX to produce H2O2 instead of H2O in order for it to become
#Balanced.
print('NGAM NADPHOX mass balance: ', ngam_nadphox_c.check_mass_balance())

NGAM NADPHOX mass balance:  {}


In [8]:
#Add constraints such that the NADPH oxidase flux ratios are equally distributed in each compartment
#Methodology is the same as the one by Cheung et al (2014) along with other studies stemming
#from it (Shameer et al 2018, Topfer et al 2020)

nadphox_c_s = mm.set_fix_flux_ratio({ngam_nadphox_c.id:1, ngam_nadphox_s.id:1},ios2164_orig)
nadphox_c_s.name = "nadphox_cs_ratio"
nadphox_s_m = mm.set_fix_flux_ratio({ngam_nadphox_s.id:1, ngam_nadphox_m.id:1}, ios2164_orig)
nadphox_s_m.name = "nadphox_sm_ratio"

ios2164_orig.add_cons_vars(nadphox_c_s)
ios2164_orig.add_cons_vars(nadphox_s_m)



In [9]:
#Does not work
#Add constraint such that ATP:NADPH oxidase ratio is at 3:1

fex_nadphoxc =  mm.get_flux_exp(ios2164_orig, ngam_nadphox_c)
fex_nadphoxs =  mm.get_flux_exp(ios2164_orig, ngam_nadphox_s)
fex_nadphoxm =  mm.get_flux_exp(ios2164_orig, ngam_nadphox_m)
fex_atpc = mm.get_flux_exp(ios2164_orig, ngam_atp_c)
fex_atps =  mm.get_flux_exp(ios2164_orig, ngam_atp_s)
fex_atpm = mm.get_flux_exp(ios2164_orig, ngam_atp_m)
fex_atpx =mm.get_flux_exp(ios2164_orig, ngam_atp_x)


nadphox_atpase = ios2164_orig.problem.Constraint(3*(fex_nadphoxc+fex_nadphoxs +fex_nadphoxm) 
                                     - 1*(fex_atpc + fex_atps + fex_atpm),
                                     lb=0,ub=0)
nadphox_atpase.name = "nadphox_atpase_ratio"


ios2164_orig.add_cons_vars(nadphox_atpase)
ios2164_orig.constraints.nadphox_cs_ratio.expression

-1.0*ngam_nadphox_c + 1.0*ngam_nadphox_c_reverse_4016e + 1.0*ngam_nadphox_s - 1.0*ngam_nadphox_s_reverse_1b6e8

In [10]:
#Restrict all reactions to white light only
for lights in ios2164_orig.reactions:
    if 'PRISM' in lights.id:
        ios2164_orig.reactions.get_by_id(lights.id).bounds = (-inf,inf)
        #Constrain all non-white light reactions to 0
        if 'white' not in lights.id:
            ios2164_orig.reactions.get_by_id(lights.id).bounds = (0,0)


In [11]:
#Add Rubisco Oxygenase-specific reaction to model

#First retrieve the carboxylase reaction
rbc_rxn = ios2164_orig.reactions.RBPCs


#Define reaction. Id is RBPOs
rbo_rxn = Reaction('RBPOs')
rbo_rxn.name = 'Ribulose-biphosphate Oxygenase'

#Components of oxygenase reaction are as follows
#o2[s] + h2o[s] + rb15bp[s] -> 3pg[s] + 2pglyc[s]
#Interestingly, Arnold's model accounts for the generation of protons by the rubisco reaction.
#There's also support from literature on how protonation may also enhance CO2 capture by a plant Cell.
#To accomodate this I'll just add the reaction specifically for both RBO and RBC reactions.

#Retrieve metabolites
o2_s = ios2164_orig.metabolites.o2_s
h2o_s = ios2164_orig.metabolites.h2o_s
rb15bp_s = ios2164_orig.metabolites.rb15bp_s
h_s = ios2164_orig.metabolites.h_s
_3pg_s = ios2164_orig.metabolites.get_by_id('3pg_s')
_2pglyc_s = ios2164_orig.metabolites.get_by_id('2pglyc_s')

#Define Stoichiometry
rbo_rxn.add_metabolites({o2_s:-1.0, rb15bp_s:-1, h_s:2, _3pg_s: 1, _2pglyc_s: 1})

#Copy other details from rbc_rxn
rbo_rxn.subsystem = rbc_rxn.subsystem
rbo_rxn.bounds = rbc_rxn.bounds
#Define encoded genes; copied from ios2164 supplementary
rbo_rxn.gene_reaction_rule = rbc_rxn.gene_name_reaction_rule
rbo_rxn.notes['SUBSYSTEM'] = 'Calvin cycle'
#Add to model
ios2164_orig.add_reactions([rbo_rxn])

ios2164_orig.repair

ios2164_orig.reactions.RBPOs
        
#Balanced equation is as follows:
#o2_s + rb15bp_s --> 2pglyc_s + 3pg_s + 2 h_s

Reaction identifier,RBPOs
Name,Ribulose-biphosphate Oxygenase
Memory address,0x7ff52ecd8880
Stoichiometry,"o2_s + rb15bp_s --> 2pglyc_s + 3pg_s + 2 h_s O2 + D-Ribulose 1,5-bisphosphate --> 2-Phosphoglycolate + 3-Phospho-D-glycerate + 2 H+"
GPR,(LOC_Os01g58020 and LOC_Os05g35330 and LOC_Os11g32770 and LOC_Os12g10580 and LOC_Os10g21280) and...
Lower bound,0.0
Upper bound,1000000.0


In [12]:
def rebalance_model(model):
    '''
    
    This function is used to rebalance any reactions
    found in the old model to reduce any possible futile cycling. 
    
    This curation step is done with the aid of Metacyc and BRENDA
    as well as the check_mass_balance() function from CobraPy
    to validate any inconsistent chemical equations as well as
    compound formulas that weren't encoded properly. 
    
    The only remaining reactions that weren't curated were as  follows:
    - reactions with undefined side groups encoded as "R" and were therefore expectedly
    unbalanced since CobraPy does not detect them
    - Charge imbalances that were validated to be balanced based on the databases above as well as
     w/ other extant curated models
    - Spectral decomposition reactions w/c were expected to drain out of the system w/ excess photon input
    - Other reactions that are not expected to carry flux with the model constraints.

    Overall this rebalancing step reduces the amount of unbalanced internal reactions from 28% to ~13% and removes.
    - Removes any excess fluxes towards proton generating reactions when using pFBA
    - Prevents spurious Biomass generation with O2 input
    
    
    '''
    ios2164_orig.metabolites.get_by_id('4c2me_s').formula = 'C14H23N3O14P2'
    ios2164_orig.metabolites.udprmn_c.formula = 'C15H22N2O16P2' #Works
    ios2164_orig.metabolites.cgly_c.formula = 'C5H10N2O3S' #Not encoded properly
    ios2164_orig.metabolites.cholp_r.formula = 'C5H13NO4P'
    ios2164_orig.metabolites.cholp_s.formula = 'C5H13NO4P'
    ios2164_orig.metabolites.dtmp_s.formula = 'C10H13N2O8P' #Was not encoded properly in model
    ios2164_orig.metabolites.get_by_id('5dpmev_c').formula = 'C6H10O10P2'
    ios2164_orig.metabolites.lnlncoa_r.formula = 'C39H62N7O17P3S'
    ios2164_orig.metabolites.lnlnlACP_s.formula = 'C29H52N2O8PRS'
    ios2164_orig.metabolites.lnlnl_c.formula = 'C18H29O2'
    ios2164_orig.metabolites.lnlnl_s.formula = 'C18H29O2'
    ios2164_orig.metabolites.lnlnl_x.formula = 'C18H29O2'
    ios2164_orig.metabolites.o2_u.formula = 'O2' #Metabolite Not encoded properly in model
    ios2164_orig.metabolites.plast_s.formula = 'C53H82O2'
    
    
    ios2164_orig.reactions.AASADc.add_metabolites({'h_c':-1})
    ios2164_orig.reactions.AATCm.add_metabolites({'h_m':-1})
    ios2164_orig.reactions.AATCs.add_metabolites({'h_s':-1})
    ios2164_orig.reactions.ACDOc.add_metabolites({'h_c':1})
    ios2164_orig.reactions.ACGAMKc.add_metabolites({'h_c':1})
    ios2164_orig.reactions.ACHBSs.add_metabolites({'h_s':1})
    ios2164_orig.reactions.ACHYSLc.add_metabolites({'h_c':1})
    ios2164_orig.reactions.ACRSPc.add_metabolites({'h_c':-1})
    ios2164_orig.reactions.ACSs.add_metabolites({'h_s':1})
    ios2164_orig.reactions.ADEDAc.add_metabolites({'h_c':-1})
    ios2164_orig.reactions.ADMATs.add_metabolites({'h_s':1})
    ios2164_orig.reactions.ADPARTs.add_metabolites({'h_s':1})
    ios2164_orig.reactions.ALATm.add_metabolites({'h_m':1})
    ios2164_orig.reactions.ALATs.add_metabolites({'h_s':1})
    ios2164_orig.reactions.ALTNDAc.add_metabolites({'h_c':-2.0})
    ios2164_orig.reactions.AMETAHYCSTc.add_metabolites({'h_c':1,  'h2o_c':1})
    ios2164_orig.reactions.ANTPPTs.add_metabolites({'h_s':1})
    ios2164_orig.reactions.ANXANDEOs.add_metabolites({'h_s':2})
    ios2164_orig.reactions.AOBUTDx.add_metabolites({'h_x':1})
    ios2164_orig.reactions.APPRTs.add_metabolites({'h_s':1})
    ios2164_orig.reactions.ARGSSs.add_metabolites({'h_s':1})
    ios2164_orig.reactions.ASCOXc.add_metabolites({'h_c':8})
    ios2164_orig.reactions.ASCPO1c.add_metabolites({'h_c':4})
    ios2164_orig.reactions.ASCPO1s.add_metabolites({'h_s':2})
    ios2164_orig.reactions.ASCPO2c.add_metabolites({'h_c':1})
    ios2164_orig.reactions.ASCPO2s.add_metabolites({'h_s':3})
    ios2164_orig.reactions.ASNS1s.add_metabolites({'h_s':1})
    ios2164_orig.reactions.ASNTLs.add_metabolites({'h_s':1})
    ios2164_orig.reactions.ASNtm.add_metabolites({'h_m':1})
    ios2164_orig.reactions.ASPALs.add_metabolites({'h_s':1})
    ios2164_orig.reactions.ASPCLc.add_metabolites({'h_c': -1})
    ios2164_orig.reactions.ASPCT1c.add_metabolites({'h_c':1})
    ios2164_orig.reactions.ASPCTs.add_metabolites({'h_s':1})
    ios2164_orig.reactions.ASPHGNAHr.add_metabolites({'h_r':-1})
    ios2164_orig.reactions.ASPHGNAr.add_metabolites({'h_r':-1})
    ios2164_orig.reactions.ASPTLm.add_metabolites({'h_m':1})
    ios2164_orig.reactions.ASPTLs.add_metabolites({'h_s':1})
    ios2164_orig.reactions.ATAHc.add_metabolites({'h_c':1})
    ios2164_orig.reactions.ATPAPc.reaction = 'atp_c + 2.0 h2o_c --> amp_c + 2.0 pi_c + 2.0 h_c'
    ios2164_orig.reactions.ATPSs.add_metabolites({'h_u':-1})
    ios2164_orig.reactions.BRS2c.add_metabolites({'h_c':-1})
    ios2164_orig.reactions.C3STDH2c.add_metabolites({'h_c':-1})
    ios2164_orig.reactions.C4STMO1c.add_metabolites({'h_c':1})
    ios2164_orig.reactions.CAFFMTc.add_metabolites({'h_c':1})
    ios2164_orig.reactions.CAVNS2s.add_metabolites({'h_s':2})
    ios2164_orig.reactions.CBMKs.add_metabolites({'h_s':1})
    ios2164_orig.reactions.CBPSs.add_metabolites({'h_s':3})
    ios2164_orig.reactions.CC3H1c.add_metabolites({'h_c':-1})
    ios2164_orig.reactions.CDPDAGSm.add_metabolites({'h_m':1})
    ios2164_orig.reactions.CELLSc.add_metabolites({'h_c': 2})
    ios2164_orig.reactions.CHALIc.add_metabolites({'h_c':-1})
    ios2164_orig.reactions.CHOLPCTs.add_metabolites({'h_s':1})
    ios2164_orig.reactions.CKDH1c.add_metabolites({'h_c':1})
    ios2164_orig.reactions.CKDH2c.add_metabolites({'h_c':1})
    ios2164_orig.reactions.CKDH3c.add_metabolites({'h_c':1})
    ios2164_orig.reactions.CKGT1c.add_metabolites({'h_c':1})
    ios2164_orig.reactions.CKGT2c.add_metabolites({'h_c':1})
    ios2164_orig.reactions.CKGT3c.add_metabolites({'h_c':1})
    ios2164_orig.reactions.CKGT4c.add_metabolites({'h_c':1})
    ios2164_orig.reactions.CKGT5c.add_metabolites({'h_c':1})
    ios2164_orig.reactions.CKGT6c.add_metabolites({'h_c':1})
    ios2164_orig.reactions.CKGT7c.add_metabolites({'h_c':1})
    ios2164_orig.reactions.CKGT8c.add_metabolites({'h_c':1})
    ios2164_orig.reactions.CKOGT1c.add_metabolites({'h_c':1})
    ios2164_orig.reactions.CKOGT2c.add_metabolites({'h_c':1})
    ios2164_orig.reactions.CKOGT3c.add_metabolites({'h_c':1})
    ios2164_orig.reactions.CKPc.add_metabolites({'h_c':2})
    ios2164_orig.reactions.CMPAc.add_metabolites({'h_c':-1})
#     ios2164_orig.reactions.CNDNG3Gc.add_metabolites({'h_c': 2})
    ios2164_orig.reactions.CNDNG5Gc.add_metabolites({'h_c':2})
    ios2164_orig.reactions.CNDNGLT5c.add_metabolites({'h_c':2})
    ios2164_orig.reactions.COUMGUTFc.add_metabolites({'h_c':-1})
    ios2164_orig.reactions.COUMNSc.add_metabolites({'h_c':1})
    ios2164_orig.reactions.CTPCHc.add_metabolites({'h_c':-1})
    ios2164_orig.reactions.CYSSs.add_metabolites({'h_s':-1})
    ios2164_orig.reactions.CYSTLm.add_metabolites({'h_m':-3})
    ios2164_orig.reactions.CYTTHs.bounds = (-inf, 0)
    ios2164_orig.reactions.DAGLc.add_metabolites({'h_c':-1})
    ios2164_orig.reactions.DCTPDAc.add_metabolites({'h_c':-1})
    ios2164_orig.reactions.DGSNTTPc.add_metabolites({'h_c':-2})
    ios2164_orig.reactions.DHDHc.add_metabolites({'h_c':2})
    ios2164_orig.reactions.DHDPSs.add_metabolites({'h_s': -2})
    ios2164_orig.reactions.DHKFORc.add_metabolites({'h_c':1})
    ios2164_orig.reactions.DHKRc.add_metabolites({'h_c': 1})
    ios2164_orig.reactions.DHNCOAHs.add_metabolites({'h_s': 1})
    ios2164_orig.reactions.DHNCOASs.add_metabolites({'h_s': -1})
    ios2164_orig.reactions.DHNOPTs.add_metabolites({'h_s': 2})
    ios2164_orig.reactions.DHNPPTs.add_metabolites({'h_s':2})
    ios2164_orig.reactions.DHORSts.add_metabolites({'h_s':-1})
    ios2164_orig.reactions.DHPDc.add_metabolites({'h_c':-1})
    ios2164_orig.reactions.DHPRs.add_metabolites({'h_s': 2})
    ios2164_orig.reactions.DHPRys.add_metabolites({'h_s': 2})
    ios2164_orig.reactions.DHPSm.add_metabolites({'h_m':1})
    ios2164_orig.reactions.DHQORc.add_metabolites({'h_c':1})
    ios2164_orig.reactions.DHQRc.add_metabolites({'h2o_c':-1})
    ios2164_orig.reactions.DMATTs.add_metabolites({'h_s':1})
    ios2164_orig.reactions.DMATTm.add_metabolites({'h_m':1})
    ios2164_orig.reactions.DHPAHs.add_metabolites({'h_s':-1})
    ios2164_orig.reactions.DMPPTs.add_metabolites({'h_s': 1})
    ios2164_orig.reactions.DMPQMTs.add_metabolites({'h_s': 1})
    ios2164_orig.reactions.DNTPPAc.add_metabolites({'h_c':1})
    ios2164_orig.reactions.EAR160ym.add_metabolites({'h_m':-1})
    ios2164_orig.reactions.EAR160ys.add_metabolites({'h_s':-1})
    ios2164_orig.reactions.ECASDATEO1c.add_metabolites({'h_c':-1})
    ios2164_orig.reactions.ECASDATEO3c.add_metabolites({'h_c':-1})
    ios2164_orig.reactions.ECASDO1c.add_metabolites({'h_c':-1})
    ios2164_orig.reactions.ECASDO3c.add_metabolites({'h_c':-1})
    ios2164_orig.reactions.ECASDSs.add_metabolites({'h_s':1})
    ios2164_orig.reactions.EIKAUROc.add_metabolites({'h_c':-1})
    ios2164_orig.reactions.EIKAURSs.add_metabolites({'h_s':1})
    ios2164_orig.reactions.EKAURSs.add_metabolites({'h_s':1})
    ios2164_orig.reactions.ENTBSs.delete #Remove bacterial reaction w/ no GPR
    ios2164_orig.reactions.ESANDO1c.add_metabolites({'h_c':-1})
    ios2164_orig.reactions.ESANDSs.add_metabolites({'h_s':-1})
    ios2164_orig.reactions.ESANDSs.add_metabolites({'h_s':1})
    ios2164_orig.reactions.ETHAPTr.add_metabolites({'h_r':1})
    ios2164_orig.reactions.FA140COAabcx.add_metabolites({'h2o_x':-1,'h_x':1})
    ios2164_orig.reactions.FA160ACOASc.add_metabolites({'h_s':-1})
    ios2164_orig.reactions.FA160ACPHs.add_metabolites({'h_s':1})
    ios2164_orig.reactions.FA160COAabcx.add_metabolites({'h2o_x':-1,'h_x':1})
    ios2164_orig.reactions.FA161COAabcx.add_metabolites({'h2o_x':-1,'h_x':1})
    ios2164_orig.reactions.FA180COAabcx.add_metabolites({'h2o_x':-1,'h_x':1})
    ios2164_orig.reactions.FA181COAabcx.add_metabolites({'h2o_x':-1,'h_x':1})
    ios2164_orig.reactions.FA182ACOASc.add_metabolites({'h_c':-1})
    ios2164_orig.reactions.FA182ACPHs.add_metabolites({'h_s':1})
    ios2164_orig.reactions.FA182COAabcx.add_metabolites({'h2o_x':-1,'h_x':1})
    ios2164_orig.reactions.FA183ACOASc.add_metabolites({'h_c':-1})
    ios2164_orig.reactions.FA183ACPHs.add_metabolites({'h_s':1})
    ios2164_orig.reactions.FA183COAabcx.add_metabolites({'h2o_x':-1,'h_x':1})
    ios2164_orig.reactions.FA200COAHr.add_metabolites({'h_r':1})
    ios2164_orig.reactions.FA220COAHr.add_metabolites({'h_r':1})
    ios2164_orig.reactions.FA240COAHr.add_metabolites({'h_r':1})
    ios2164_orig.reactions.FA260COAHr.add_metabolites({'h_r':1})
    ios2164_orig.reactions.FA280COAHr.add_metabolites({'h_r':1})
    ios2164_orig.reactions.FA300COAHr.add_metabolites({'h_r':1})
    ios2164_orig.reactions.FAS161COA_Lc.add_metabolites({'h_c':-1})
    ios2164_orig.reactions.FAS181ACP_Lm.add_metabolites({'h_m':-1})
    ios2164_orig.reactions.FAS181ACP_Ls.add_metabolites({'h_s':-1})
    ios2164_orig.reactions.FAS181COA_Lc.add_metabolites({'h_c':-1})
    ios2164_orig.reactions.FAS182ACP_Ls.add_metabolites({'h_s':-2})
    ios2164_orig.reactions.FAS182COA_Lc.add_metabolites({'h_c':-1})
    ios2164_orig.reactions.FAS183ACP_Ls.add_metabolites({'h_s':-1})
    ios2164_orig.reactions.FAS183COA_Lc.add_metabolites({'h_c':-1})
    ios2164_orig.reactions.FERHc.add_metabolites({'h_c':-1})
    ios2164_orig.reactions.FGFTm.add_metabolites({'h_m':1})
    ios2164_orig.reactions.FGFTs.add_metabolites({'h_s':1})
    ios2164_orig.reactions.FMNATc.add_metabolites({'h_c':1})
    ios2164_orig.reactions.FMNATm.add_metabolites({'h_m':2})
    ios2164_orig.reactions.FMNATs.add_metabolites({'h_s':2})
    ios2164_orig.reactions.FNORs.add_metabolites({'h_s':2})
    ios2164_orig.reactions.FORMHLc.add_metabolites({'h_c':1})
    ios2164_orig.reactions.FRDPFTs.add_metabolites({'h_s': 5})
    ios2164_orig.reactions.FRPTc.add_metabolites({'h_c':1})
    ios2164_orig.reactions.FRTTs.add_metabolites({'h_s':1})
    ios2164_orig.reactions.FTHFCLc.add_metabolites({'h_c':1})
    ios2164_orig.reactions.FTHFCLm.add_metabolites({'h_m':1})
    ios2164_orig.reactions.FTHFCLs.add_metabolites({'h_c':1})
    ios2164_orig.reactions.GA4DOc.add_metabolites({'h2o_c':1, 'h_c':1})
    ios2164_orig.reactions.GAAKORG10c.add_metabolites({'h_c':-1})
    ios2164_orig.reactions.GAAKORG11c.add_metabolites({'h2o_c':1})
    ios2164_orig.reactions.GAAKORG12c.add_metabolites({'h_c':-1})
    ios2164_orig.reactions.GAAKORG8c.add_metabolites({'h_c':-1})
    ios2164_orig.reactions.GAAKORG9c.add_metabolites({'h2o_c':1})
    ios2164_orig.reactions.GAMPTc.add_metabolites({'h_c':1.0})
    ios2164_orig.reactions.GBDPs.add_metabolites({'h_s':1})
    ios2164_orig.reactions.GCOADCx.add_metabolites({'h_x':-1})
    ios2164_orig.reactions.GDPMDHc.add_metabolites({'h_c':-1})
    ios2164_orig.reactions.GENT2c.add_metabolites({'h_c':1})
    ios2164_orig.reactions.GENT3c.add_metabolites({'h_c':1})
    ios2164_orig.reactions.GLCNSc.add_metabolites({'h_c':2})
    ios2164_orig.reactions.GLUTRSs.add_metabolites({'h_s':1})
    ios2164_orig.reactions.GLYATx.add_metabolites({'h_x':-1})
    ios2164_orig.reactions.GLYB1s.add_metabolites({'h_s':2})
    ios2164_orig.reactions.GLYDHDm.add_metabolites({'h_m':-1})
    ios2164_orig.reactions.GLYTLm.add_metabolites({'h_m':1})
    ios2164_orig.reactions.GLYTLs.add_metabolites({'h_m':1})
    ios2164_orig.reactions.GMPc.add_metabolites({'h_c':1})
    ios2164_orig.reactions.GNSKc.add_metabolites({'h_c':-1})
    ios2164_orig.reactions.GRTTm.add_metabolites({'h_m':1})
    ios2164_orig.reactions.GRTTs.add_metabolites({'h_s':1})
    ios2164_orig.reactions.GSNPc.add_metabolites({'h_c':-1})
    ios2164_orig.reactions.GSNRHc.add_metabolites({'h_c':-1})
    ios2164_orig.reactions.GTHDHc.add_metabolites({'h_c':-1})
    ios2164_orig.reactions.GTHDHs.add_metabolites({'h_s':-1})
    ios2164_orig.reactions.GTPCYTDPTc.add_metabolites({'h_c':1})
    ios2164_orig.reactions.GTPCYTDPTs.add_metabolites({'h_s':1})
    ios2164_orig.reactions.GTPDHs.add_metabolites({'h_s':1})
    ios2164_orig.reactions.GTPDPKs.add_metabolites({'h_s':1})
    ios2164_orig.reactions.H2Sts.add_metabolites({'h_s':1})
    ios2164_orig.reactions.HCYSMT2c.add_metabolites({'h_c':1})
    ios2164_orig.reactions.HDHs.add_metabolites({'h_s': -1})
    ios2164_orig.reactions.HEMEOSm.add_metabolites({'h_m':1})
    ios2164_orig.reactions.HFERMTc.add_metabolites({'h_c':1})
    ios2164_orig.reactions.HISTLm.add_metabolites({'h_m':1})
    ios2164_orig.reactions.HISTLs.add_metabolites({'h_s':1})
    ios2164_orig.reactions.HMGCOARc.add_metabolites({'h_c': -1.0})
    ios2164_orig.reactions.HMGCOARyc.add_metabolites({'h_c': -1.0})
    ios2164_orig.reactions.HMGLm.add_metabolites({'h_m':1})
    ios2164_orig.reactions.HOXGs.add_metabolites({'h_s':-2})
    ios2164_orig.reactions.HPPKm.add_metabolites({'h_m':1})
    ios2164_orig.reactions.IAAAMYHc.add_metabolites({'h_c':1})
    ios2164_orig.reactions.IAAIASc.add_metabolites({'h_c':1})
    ios2164_orig.reactions.IAAIGSc.add_metabolites({'h_c':1})
    ios2164_orig.reactions.ILETLm.add_metabolites({'h_m':1})
    ios2164_orig.reactions.ILETLs.add_metabolites({'h_s':1})
    ios2164_orig.reactions.IPDPT1s.add_metabolites({'h_s':2})
    ios2164_orig.reactions.IPDPT2s.add_metabolites({'h_s':1})
    ios2164_orig.reactions.IPKK2c.add_metabolites({'h_c':-2})
    ios2164_orig.reactions.IPKK4c.add_metabolites({'h_c':-2})
    ios2164_orig.reactions.IPKK5c.add_metabolites({'h_c':-2})
    ios2164_orig.reactions.IPKK6c.add_metabolites({'h_c': -2.0})
    ios2164_orig.reactions.ITK1Kc.add_metabolites({'h_c':-2})
    ios2164_orig.reactions.LCYCORc.add_metabolites({'h_c': 1})
    ios2164_orig.reactions.LCYSTSm.add_metabolites({'h_m':2})
    ios2164_orig.reactions.LCYSTSs.add_metabolites({'h_s':2})
    ios2164_orig.reactions.LIMSs.add_metabolites({'h_s':1})
    ios2164_orig.reactions.LNL9HPLs.add_metabolites({'h_s':-2})
    ios2164_orig.reactions.LPLGDOc.add_metabolites({'h_c':1})
    ios2164_orig.reactions.LYSTLm.add_metabolites({'h_m':1})
    ios2164_orig.reactions.LYSTLs.add_metabolites({'h_s':1})
    ios2164_orig.reactions.MACPCLm.add_metabolites({'h_m':-2})
    ios2164_orig.reactions.MACPCLs.add_metabolites({'h_s':-1})
    ios2164_orig.reactions.MACPMMEc.add_metabolites({'h_c':-1})
    ios2164_orig.reactions.MAGLc.add_metabolites({'h_c':-1})
    ios2164_orig.reactions.MANKINc.add_metabolites({'h_c':1})
    ios2164_orig.reactions.MARMSc.add_metabolites({'h_c':-1})
    ios2164_orig.reactions.MCOATAc.add_metabolites({'h_c':1})
    ios2164_orig.reactions.MCOATAm.add_metabolites({'h_m':1})
    ios2164_orig.reactions.MCOATAs.add_metabolites({'h_s':1})
    ios2164_orig.reactions.MDHARm.add_metabolites({'h_m':-2})
    ios2164_orig.reactions.MDHARc.add_metabolites({'h_c':-2})
    ios2164_orig.reactions.MDHARyc.add_metabolites({'h_c':-2})
    ios2164_orig.reactions.MDHARym.add_metabolites({'h_m':-2})
    ios2164_orig.reactions.MECDPDH5s.add_metabolites({'h_s':2})
    ios2164_orig.reactions.MEPCTs.add_metabolites({'h_s':1})
    ios2164_orig.reactions.METSc.add_metabolites({'h_c':1.0})
    ios2164_orig.reactions.METSs.add_metabolites({'h_s':1})
    ios2164_orig.reactions.METTLm.add_metabolites({'h_m':1})
    ios2164_orig.reactions.METTLs.add_metabolites({'h_s':1})
    ios2164_orig.reactions.MGCHLs.add_metabolites({'h_s': 2})
    ios2164_orig.reactions.MGCm.add_metabolites({'h_m':1})
    ios2164_orig.reactions.MI1345PP2c.add_metabolites({'h_c':-2})
    ios2164_orig.reactions.MI1345PPc.add_metabolites({'h_c': -2.0})
    ios2164_orig.reactions.MI1346PK1c.add_metabolites({'h_c':-2})
    ios2164_orig.reactions.MI1346PK2c.add_metabolites({'h_c':-2})
    ios2164_orig.reactions.MI1456PKc.add_metabolites({'h_c':-2})
    ios2164_orig.reactions.MI34Pc.add_metabolites({'h_c':1})
    ios2164_orig.reactions.MLTHFR_Fc.add_metabolites({'h_c':3})
    ios2164_orig.reactions.MLTHFRc.add_metabolites({'h_c':-1})
    ios2164_orig.reactions.MLTHFRs.add_metabolites({'h_s':-1})
    ios2164_orig.reactions.MOMASs.add_metabolites({'h_s':-1})
    ios2164_orig.reactions.MOMASys.add_metabolites({'h_s':-1})
    ios2164_orig.reactions.MPMECY1s.add_metabolites({'h_s':-1})
    ios2164_orig.reactions.MPMECY2s.add_metabolites({'h_s':2})
    ios2164_orig.reactions.MPMTs.add_metabolites({'h_s': -1})
    ios2164_orig.reactions.MRs.add_metabolites({'h_s':2})
    ios2164_orig.reactions.MSBQMTs.add_metabolites({'h_s':1})
    ios2164_orig.reactions.MTAMnh4m.add_metabolites({'h_m':1})
    ios2164_orig.reactions.NADOR2u.add_metabolites({'h_u':-1.0})
    ios2164_orig.reactions.NNAMc.add_metabolites({'h_c':-1})
    ios2164_orig.reactions.NMNDAc.add_metabolites({'h_c':-1})
    ios2164_orig.reactions.NNDPRs.add_metabolites({'h_s':1})
    ios2164_orig.reactions.NO3Rm.add_metabolites({'h_m':-2})
    ios2164_orig.reactions.NOSc.reaction = '2 arg-L_c + 3 nadph_c + 4 o2_c + 1 h_c --> 2 citr-L_c + 3 nadp_c + 2 no_c + 4 h2o_c'
    ios2164_orig.reactions.NRGNAORc.add_metabolites({'h_c':1})
    ios2164_orig.reactions.NRGNMTc.add_metabolites({'h_c':1})
    ios2164_orig.reactions.NRGNNORc.add_metabolites({'h_c':1})
    ios2164_orig.reactions.OCTDPOTs.add_metabolites({'h_s': 1})
    ios2164_orig.reactions.ORPRTs.add_metabolites({'h_s':-1})
    ios2164_orig.reactions.OXAACc.add_metabolites({'h_c':-1})
    ios2164_orig.reactions.P450MO1c.add_metabolites({'h_c':- 1})
    ios2164_orig.reactions.P450MO2c.add_metabolites({'h_c':-1})
    ios2164_orig.reactions.P450MO3c.add_metabolites({'h_c':-1})
    ios2164_orig.reactions.P450MO4c.add_metabolites({'h_c':-1})
    ios2164_orig.reactions.P450MO5c.add_metabolites({'h_c':-1})
    ios2164_orig.reactions.P450MO6c.add_metabolites({'h_c':-1})
    ios2164_orig.reactions.PCAHr.add_metabolites({'h_r':-1})
    ios2164_orig.reactions.PEDCm.add_metabolites({'h_m':1})
    ios2164_orig.reactions.PEDCr.add_metabolites({'h_r':1})
    ios2164_orig.reactions.PEPSs.add_metabolites({'h_s':2})
    ios2164_orig.reactions.PHENH4Lc.add_metabolites({'h_c':-1})
    ios2164_orig.reactions.PHETLm.add_metabolites({'h_m':1})
    ios2164_orig.reactions.PHETLs.add_metabolites({'h_s':1}) 
    ios2164_orig.reactions.PHYTS1s.add_metabolites({'h_s':1})
    ios2164_orig.reactions.PHYTS2s.add_metabolites({'h_s':1})
    ios2164_orig.reactions.PINCLc.add_metabolites({'h_c':-1})
    ios2164_orig.reactions.PLIBTNSm.add_metabolites({'h_m':-3})
    ios2164_orig.reactions.PLIVLPNSm.add_metabolites({'h_m':-3})
    ios2164_orig.reactions.PPASm.add_metabolites({'h_m':-1})
    ios2164_orig.reactions.PPASs.add_metabolites({'h_s':-1})
    ios2164_orig.reactions.PPDKc.add_metabolites({'h_c':1})
    ios2164_orig.reactions.PPDKs.add_metabolites({'h_s':2})
    ios2164_orig.reactions.PPPGPPPc.add_metabolites({'h_c':1})
    ios2164_orig.reactions.PRADPs.add_metabolites({'h_s': 1})
    ios2164_orig.reactions.PRAICs.add_metabolites({'h_s':-1})
    ios2164_orig.reactions.PRDPARs.add_metabolites({'h_s':1})
    ios2164_orig.reactions.PRFGSs.add_metabolites({'h_s':1})
    ios2164_orig.reactions.PRIMADNSs.add_metabolites({'h_s':1})
    ios2164_orig.reactions.PRIMARTSc.add_metabolites({'h_s':2})
    ios2164_orig.reactions.PROTLm.add_metabolites({'h_m':1})
    ios2164_orig.reactions.PROTLs.add_metabolites({'h_s':1})
    ios2164_orig.reactions.QRCTNGLT3c.add_metabolites({'h_c': 1})
    ios2164_orig.reactions.RBPCs.add_metabolites({'h_s':2})
    ios2164_orig.reactions.RIBFKc.add_metabolites({'h_c':-1})
    ios2164_orig.reactions.RIBFSs.add_metabolites({'h_s':2})
    ios2164_orig.reactions.SELNPS.add_metabolites({'h_c':-1})
    ios2164_orig.reactions.SERTLm.add_metabolites({'h_m':1})
    ios2164_orig.reactions.SERTLs.add_metabolites({'h_s':1})
    ios2164_orig.reactions.SHCHFs.add_metabolites({'h_s':3})
    ios2164_orig.reactions.SOXs.add_metabolites({'h_s':1})
    ios2164_orig.reactions.SQTs.add_metabolites({'h_s':1})
    ios2164_orig.reactions.STEMOc.add_metabolites({'h_c': -1})
    ios2164_orig.reactions.STEMSs.add_metabolites({'h_s': 1})
    ios2164_orig.reactions.SUATs.add_metabolites({'h_s':1})
    ios2164_orig.reactions.SULRs.add_metabolites({'h_s':6})
    ios2164_orig.reactions.TAGLc.add_metabolites({'h_c':-1})
    ios2164_orig.reactions.THR3DHx.add_metabolites({'h_x':-1.0})
    ios2164_orig.reactions.THRTLm.add_metabolites({'h_m':1})
    ios2164_orig.reactions.TMDPKc.add_metabolites({'h_c':1})
    ios2164_orig.reactions.TMPPPs.add_metabolites({'h_s':1})
    ios2164_orig.reactions.TRICINS1c.add_metabolites({'h_c':1})
    ios2164_orig.reactions.TRICINS2c.add_metabolites({'h_c':1})
    ios2164_orig.reactions.TRPTLm.add_metabolites({'h_m':1})
    ios2164_orig.reactions.TRPTLs.add_metabolites({'h_s':1})
    ios2164_orig.reactions.TYRLs.add_metabolites({'h_s':1})
    ios2164_orig.reactions.TYRNH4Lc.add_metabolites({'h_c':-1})
    ios2164_orig.reactions.TYRTLm.add_metabolites({'h_m':1})
    ios2164_orig.reactions.TYRTLs.add_metabolites({'h_s':1})
    ios2164_orig.reactions.UAMASc.add_metabolites({'h_c':1})
    ios2164_orig.reactions.UDPGALMc.add_metabolites({'h_c':-2})
    ios2164_orig.reactions.UDPGALts.add_metabolites({'h_s':1})
    ios2164_orig.reactions.UDPGATc.add_metabolites({'h_c':1})
    ios2164_orig.reactions.UDPGts.add_metabolites({'h_s':1})
    ios2164_orig.reactions.UDPSQSs.add_metabolites({'h_s':-1})
    ios2164_orig.reactions.UPRTs.add_metabolites({'h_s':-1})
    ios2164_orig.reactions.URDGLAHc.add_metabolites({'h_c':-1})     
    ios2164_orig.reactions.UREASEc.add_metabolites({'h_c':-2.0})
    ios2164_orig.reactions.UROc.add_metabolites({'h_c':-1})
    ios2164_orig.reactions.VALTLm.add_metabolites({'h_m':1})
    ios2164_orig.reactions.VALTLs.add_metabolites({'h_s':1})
    ios2164_orig.reactions.VLXANDEOs.add_metabolites({'h_s':2})
    ios2164_orig.reactions.XSc.add_metabolites({'h_c':2})
    ios2164_orig.reactions.get_by_id('2HEDBT8GTc').add_metabolites({'h_c':1})
    ios2164_orig.reactions.get_by_id('2HNDBT8GTc').add_metabolites({'h_c':1})
    ios2164_orig.reactions.get_by_id('2HNDGDHc').add_metabolites({'h_c':1})
    ios2164_orig.reactions.get_by_id('2SBZCOALs').add_metabolites({'h_s': 1})
    ios2164_orig.reactions.get_by_id('3OAS180m').add_metabolites({'h_m':1})
    ios2164_orig.reactions.get_by_id('3OAS180s').add_metabolites({'h_s':1})
    ios2164_orig.reactions.get_by_id('SUCL(ADP)m').add_metabolites({'h_m':-1})
    ios2164_orig.reactions.get_by_id('SUCL(GDP)m').add_metabolites({'h_m':-1})
    
    
    #Last iteration
    ios2164_orig.reactions.ACSERLc.add_metabolites({'h_c':-1})
    ios2164_orig.reactions.PRIMARDSs.add_metabolites({'h_s': 1})
    ios2164_orig.reactions.DLDLBTs.add_metabolites({'h_s': -1})
    ios2164_orig.reactions.PC2Ds.add_metabolites({'h_s': 2})
    ios2164_orig.reactions.GSNDc.add_metabolites({'h_c':-2})
    ios2164_orig.reactions.CYSTLs.add_metabolites({'h_c': 1})
    ios2164_orig.reactions.FRDPFTm.add_metabolites({'h_m':5})
    ios2164_orig.reactions.ARGTs.add_metabolites({'h_s':1})
    ios2164_orig.reactions.AMCPCOc.add_metabolites({'h_c':2})
    

#     ios2164_orig.metabolites.cdpdag_os_r.formula = 'C14H18N3O15P2R2'
#     ios2164_orig.metabolites.ocdcya_s.formula = 'C18H29O2' #Doesn't work

In [13]:
#Let's add a specific demand reactions for excess oxygen and protons

dm_o2 = Reaction('DM_o2(c)')
dm_o2.name = "Demand reaction for oxygen"

o2_c = ios2164_orig.metabolites.o2_c
dm_o2.add_metabolites({o2_c:-1.0})
dm_o2.bounds = (0, inf)
dm_o2.notes['SUBSYSTEM'] = 'Demand'
ios2164_orig.add_reactions([dm_o2])

#Demand reactions for protons. If needed, I can add this to the model. This was from B&B(2019) and was
#incorporated to deal with proton recycling in the model.

dm_h_c = Reaction('DM_h(c)')
dm_h_c.name = "Demand reaction for excess protons (cytosolic)"
h_c = ios2164_orig.metabolites.h_c
dm_h_c.add_metabolites({h_c:-1})
dm_h_c.bounds = (0, 0)
ios2164_orig.add_reactions([dm_h_c])

dm_h_s = Reaction('DM_h(s)')
dm_h_s.name = "Demand reaction for excess protons (stromal)"
h_s = ios2164_orig.metabolites.h_s
dm_h_s.add_metabolites({h_s:-1})
dm_h_s.bounds = (0, 0)
ios2164_orig.add_reactions([dm_h_s])

dm_h_x = Reaction('DM_h(x)')
dm_h_x.name = "Demand reaction for excess protons (Peroxisomal)"
h_x = ios2164_orig.metabolites.h_x
dm_h_x.add_metabolites({h_x:-1})
dm_h_x.bounds = (0, 0)
ios2164_orig.add_reactions([dm_h_x])

dm_h_m = Reaction('DM_h(m)')
dm_h_m.name = "Demand reaction for excess protons (mitochodrial)"
h_m = ios2164_orig.metabolites.h_m
dm_h_m.add_metabolites({h_m:-1})
dm_h_m.bounds = (0, 0)
ios2164_orig.add_reactions([dm_h_m])


dm_h_u= Reaction('DM_h(u)')
dm_h_u.name = "Demand reaction for excess protons (thylakoid)"
h_u = ios2164_orig.metabolites.h_u
dm_h_u.add_metabolites({h_u:-1})
dm_h_u.bounds = (0, 0)
ios2164_orig.add_reactions([dm_h_u])

dm_h_v= Reaction('DM_h(v)')
dm_h_v.name = "Demand reaction for excess protons (vacuole)"
h_v = ios2164_orig.metabolites.h_v
dm_h_v.add_metabolites({h_v:-1})
dm_h_v.bounds = (0, 0)
ios2164_orig.add_reactions([dm_h_v])

dm_h_r= Reaction('DM_h(r)')
dm_h_r.name = "Demand reaction for excess protons (ER)"
h_r = ios2164_orig.metabolites.h_r
dm_h_r.add_metabolites({h_r:-1})
dm_h_r.bounds = (0, 0)
ios2164_orig.add_reactions([dm_h_r])


dm_h2o_c = Reaction('DM_h2o(c)')
dm_h2o_c.name = "Demand reaction for excess H2O production"
h2o_c = ios2164_orig.metabolites.h2o_c
dm_h2o_c.add_metabolites({h2o_c:-1})
dm_h2o_c.bounds = (-inf,inf)
ios2164_orig.add_reactions([dm_h2o_c])

ios2164_orig.repair()

    

In [14]:
# #Turn off all EC -> cytosol reactions that are used by Sink tissues as these may lead to 
# #Erroneous cellular efflux of photoassimilates.
# #Maybe I can just set it as unidirectional kaya? Either way this would prevent assimilation rin


tex_reactions = [rxns for rxns in ios2164_orig.reactions if "tex" in rxns.id]
tex_reactions

sink_reactions_to_turn_off = ['SUCRtex', 'FRUtex', 'GLCtex', 'ASNtex', 'GLNtex', 'ETOHtex',
                              'ASNtex', 'ACtex', 'LACtex', 'TSULtex', 'ASNtex', 'LACtex',
                              'ALAtex', 'ARGtex', 'ASPtex', 'CYStex', 'GLUtex', 'HIStex',
                              'ILEtex', 'LEUtex', 'LYStex', 'METtex', 'PHEtex', 'PROtex', 
                              'SERtex', 'THRtex',  'TRPtex', 'TYRtex', 'VALtex']


for rxns in sink_reactions_to_turn_off:
    ios2164_orig.reactions.get_by_id(rxns).bounds = (0,inf)


#Based on Yang et al (2020), Phloem loading may occur apoplastically so this assumption may not make too much sense.
#Yang, G., Wei, Q., Huang, H., & Xia, J. (2020). Amino acid transporters in plant cells: A brief review. Plants, 9(8), 967.
#Upon reviewing this, I think the paper is only pertaining to the apoplastic transport at the level of the 
#Phloem cell

#Rebalance all reactions encoded in the above step

rebalance_model(ios2164_orig)

In [15]:
ios2164_m = copy.deepcopy(ios2164_orig)
for met in ios2164_m.metabolites:
    if met.compartment != "e": #Extracellular, defines boundary reactions
        met.id = str(met.id) + '0'
        met.compartment = str(met.compartment) + '0'

    
        
for rxn in ios2164_m.reactions:
    if "EX_" not in rxn.id: #Exclude exchange reactions
        rxn.id = str(rxn.id)  + '_M'


#Change name of reaction 
ios2164_m.repair()
ios2164_bs = copy.deepcopy(ios2164_orig)

#Rename BS reactions and metabolites
for met in ios2164_bs.metabolites:
    if met.compartment != "e": #Extracellular, defines boundary reactions
        met.id = str(met.id) + '1'
        met.compartment = str(met.compartment) + '1'
    
for rxn in ios2164_bs.reactions:
    if "EX_" not in rxn.id: #Exclude exchange reactions
        rxn.id = str(rxn.id)  + '_BS'
        
ios2164_bs.repair()

ios2164_2cell = ios2164_m.merge(ios2164_bs, inplace=True, objective = 'sum')

#Apparently compartments don't update after  merging so I've updated it na lang after that

ios2164_2cell.compartments = {
 'c0': 'M Cell Cytoplasm',
 's0': 'M Cell Plastid',
 'r0': 'M Cell Endoplasmic_reticulum',
 'm0': 'M Cell Mitochondrion',
 'e': 'Extracellular',
 'u0': 'M Cell Thylakoid',
 'x0': 'M Cell Peroxisome',
 'v0': 'M Cell Vacuole',
 'c1': 'BS Cell Cytoplasm',
 's1': 'BS Cell Plastid',
 'r1': 'BS Cell Endoplasmic_reticulum',
 'm1': 'BS Cell Mitochondrion',
 'u1': 'BS Cell Thylakoid',
 'x1': 'BS Cell Peroxisome',
 'v1': 'BS Cell Vacuole'}


#Also need to rebuild the model medium
ios2164_2cell.repair()



Read LP format model from file /tmp/tmpeujwjxio.lp
Reading time = 0.01 seconds
: 2002 rows, 4924 columns, 21402 nonzeros
Read LP format model from file /tmp/tmpojd93to4.lp
Reading time = 0.01 seconds
: 2002 rows, 4924 columns, 21402 nonzeros
Read LP format model from file /tmp/tmpjr7msp6_.lp
Reading time = 0.01 seconds
: 2002 rows, 4924 columns, 21402 nonzeros


Ignoring reaction 'EX_co2(e)' since it already exists.
Ignoring reaction 'EX_h2o(e)' since it already exists.
Ignoring reaction 'EX_h(e)' since it already exists.
Ignoring reaction 'EX_no3(e)' since it already exists.
Ignoring reaction 'EX_o2(e)' since it already exists.
Ignoring reaction 'EX_pi(e)' since it already exists.
Ignoring reaction 'EX_sucr(e)' since it already exists.
Ignoring reaction 'EX_fru-B(e)' since it already exists.
Ignoring reaction 'EX_glc-A(e)' since it already exists.
Ignoring reaction 'EX_so4(e)' since it already exists.
Ignoring reaction 'EX_hco3(e)' since it already exists.
Ignoring reaction 'EX_so3(e)' since it already exists.
Ignoring reaction 'EX_h2s(e)' since it already exists.
Ignoring reaction 'EX_nh4(e)' since it already exists.
Ignoring reaction 'EX_asn-L(e)' since it already exists.
Ignoring reaction 'EX_gln-L(e)' since it already exists.
Ignoring reaction 'EX_etoh(e)' since it already exists.
Ignoring reaction 'EX_ac(e)' since it already exists.
Igno

From Blatke & Brautigam (2019):

"The mesophyll and bundle sheath networks are connected by a range of cytosolic transport metabolites including amino acids, sugars (glucose, fructose, sucrose, trehalose, ribose), single phosphorylated sugar (glucose-6-phosphate, glucose-1-phosphate, fructose-6-phosphate, sucrose-6-phosphate), mono-/di-/tri-carboxylic acids (phosphoenolpyruvate, pyruvate, citrate, cis-aconitate, isocitrate, α-ketoglutarate, succinate, fumarate, malate), glyceric acids (2-Phosphoglycerate, 3-Phosphoglycerate), glycolate, glycerate, glyceraldehyde-3-phosphate, di-hydroxyacetone-phosphate and CO2. 

Nucleotides, NAD/NADH, NADP/NADPH, pyrophosphate, inorganic phosphate are not considered as transport metabolites. 

Oxaloacetate has been excluded as transport metabolite since concentrations of oxaloacetate are very low in vivo and it is reasonably unstable in aqueous solutions. 

Other small molecules that can be imported by the bundle sheath from the environment, as well as protons and HCO3-, are not exchanged between the two cell types.

#So bale yung mga organic na soluble lang with reasonably long cellular half-lives.

In [16]:
#Open the initial curated list to get the metabolite ids
transport_rxns = pd.read_csv('./model/initial_list_transport_rxns.csv')
#Ok it gets the reactions now

for item in ios2164_orig.metabolites:
    if item.compartment == "c":
        if item.id in transport_rxns.id.values:
            pd_rxn = Reaction()
            #Generate reaction name
            pd_rxn.id = str(item.id[:-2] + '_pd') #Replaces suffix with '_pd'
            pd_rxn.name = str('Plasmodesmatal transport of ' + item.name)
            pd_rxn.subsystem = str('PD Transport Reaction')
            #Reaction is reversible
            pd_rxn.lower_bound = -inf
            pd_rxn.upper_bound = inf
            pd_rxn.notes['SUBSYSTEM']='Plasmodesmatal Transport'
            
            #Generate metabolite names
            
            
            met_mcell = str(item.id) + '0'
            met_bcell = str(item.id) + '1'
            
            met_mcell = ios2164_2cell.metabolites.get_by_id(met_mcell)
            met_bcell = ios2164_2cell.metabolites.get_by_id(met_bcell)
            pd_rxn.add_metabolites({met_mcell:-1.0, met_bcell:1.0})
            
            
            ios2164_2cell.add_reactions([pd_rxn])
            
ios2164_2cell.repair()


#Print how many reactions added to model
printout = 'added ' + str(len(transport_rxns)) + ' PD reactions to model'
print(printout)


#Set Sucrose PD to be non-reversible (M -> BS) so as to properly model bulk flow
ios2164_2cell.reactions.get_by_id('sucr_pd').bounds = (0, inf)

#Set Gas influx is such a way that it allows one-way gas evolution from BS to M cells (from respiration)
#But should be restricted to prevent free CO2/O2 from flowing into BS cell. Without this constraint pFBA will
#Predict that the most efficient way for Rubisco fixation is to directly import CO2 from the M cell to BS cell symplastically,
#Given a BS cell with an unconstrained rubisco species. This constraint is based on the assumption that carbon fixation (either to 3PG or to other C3/C4 acids) takes place 
#in photosynthetically active tissue and that lateral diffusion of CO2 is negligible (Morison et  al., 2005)
ios2164_2cell.reactions.get_by_id('co2_pd').bounds = (-inf, 0)
ios2164_2cell.reactions.get_by_id('o2_pd').bounds = (-inf, 0)

#Turn off all NGAMs in the meantime. I can return it when I perform the  simulations directly
for rxns in ios2164_orig.reactions: 
    if 'ngam_atp' in rxns.id:
        ios2164_orig.reactions.get_by_id(rxns.id).bounds = (0,0)
        print(rxns.id, rxns.bounds)


added 73 PD reactions to model
ngam_atp_c (0, 0)
ngam_atp_s (0, 0)
ngam_atp_x (0, 0)
ngam_atp_m (0, 0)


In [17]:
#Set all external fluxes to 0 muna to make it all easy.
for rxns in ios2164_2cell.exchanges:
    rxn_id = rxns.id
    ios2164_2cell.reactions.get_by_id(rxn_id).bounds = (0, 0)


In [18]:
#Rebalance BS Phloem demand reaction to account for transitory starch accumulation (based on assumed 10:1 ratio between BS and M cell volumes)
#Starch production in M cell = 1.863 flux units Starch/unit Phloem
#Starch production in BS cell = 0.207 flux units Starch/unit plhoem

ios2164_2cell.reactions.DM_Phloem_BS.reaction = '''0.0183 ala-L_c1 + 0.0958 arg-L_c1 + 0.172 asn-L_c1 + 0.121 asp-L_c1 + 0.189 gln-L_c1 
+ 0.156 glu-L_c1 + 0.00969 his-L_c1 + 0.0204 ile-L_c1 + 0.0172 leu-L_c1 + 0.0516 lys-L_c1 + 0.0043 phe-L_c1 + 0.0129 pro-L_c1 + 0.116 ser-L_c1 
+ 1.863 starch_s0 + 0.207 starch_s1 + 0.618 sucr_c1 + 0.044 thr-L_c1 + 0.00539 tyr-L_c1 + 0.0432 val-L_c1 -->'''


In [19]:
mets = open('./misc/metabolites.csv', 'w')
rxns = open('./misc/reactions.csv', 'w')
cons = open('./misc/constraints.csv', 'w')
subsystems = open('./misc/subsystems.csv', 'w')

met_write = csv.writer(mets)
rxns_write = csv.writer(rxns)
cons_write = csv.writer(cons)
subsystems_write = csv.writer(subsystems)

for item in ios2164_2cell.metabolites:
    row = [item.id, item.name]
    met_write.writerow(row)
    

for item in ios2164_2cell.reactions:
    row = [item.id, item.name, item.build_reaction_string(),item.bounds]
    rxns_write.writerow(row)

for item  in ios2164_2cell.constraints:
    row =  [str(item.name), str(item.expression)]
    cons_write.writerow(row)
    

for subsystems in ios2164_2cell.groups:
    row = [subsystems]
    subsystems_write.writerow(row)
    


In [20]:
#Checking lang
imbalanced_rxns = list()

non_exchange_rxns = set(ios2164_orig.reactions) - set(ios2164_orig.boundary) - set(ios2164_orig.demands)
#Check mass balance of original ios2164 model

for rxns in non_exchange_rxns:
        #Retrieve mass balance dictionary
        mb_dict = rxns.check_mass_balance()
        #Check if imbalanced 
        if len(mb_dict) != 0:
            imbalanced_rxns.append(rxns)


#149 imbalanced rxns that are in boundary
#636 reactions that are within the cell. Let's check which of these are imbaalanced.
print('imbalanced reactions length for ios2164_orig after rebalancing:', len(imbalanced_rxns))
print('percent imbalanced reactions', (len(imbalanced_rxns)/len(non_exchange_rxns)) * 100)

imbalanced reactions length for ios2164_orig after rebalancing: 290
percent imbalanced reactions 12.59774109470026


In [21]:
cobra.io.write_sbml_model(ios2164_2cell, filename= "./model/ios2164_2cell.xml")
#Save Json for mapping
cobra.io.save_json_model(ios2164_2cell, filename="./model/ios2164_2cell.json")
cobra.io.save_json_model(ios2164_bs, filename="./model/ios2164_bscell.json")
cobra.io.save_json_model(ios2164_m, filename="./model/ios2164_mcell.json")                    
#Save one-cell model to incorporate updates
cobra.io.write_sbml_model(ios2164_orig, filename="./model/ios2164_1cell.xml")


#Add trans reactions then afterwards save the model to json
import model_initialize as m_i
m_i.add_trans_reactions(ios2164_2cell)
cobra.io.save_json_model(ios2164_2cell, filename= "./model/ios2164_2cell_w-trans.json")